In [5]:
import pandas as pd
import numpy as np
import nltk
from tqdm import tqdm

In [8]:
df = pd.read_csv("..\data\cleaned_data.csv")
df = df.sample(frac=1).reset_index(drop=True)
new_df = df.iloc[:1000, :]

## Vader

In [3]:
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

In [4]:
print(new_df['review_body'][0])
sia.polarity_scores(new_df['review_body'][0])

fit well clunki


{'neg': 0.0, 'neu': 0.179, 'pos': 0.821, 'compound': 0.5574}

In [5]:
res = []
print(len(new_df))
for index, row in tqdm(new_df.iterrows(), total=len(new_df)):
    text = row['review_body']
    temp = sia.polarity_scores(text)
    for key, value in temp.items():
        if value == max(temp['pos'], temp['neg'], temp['neu']):
            res.append("vader_" + key)
            break
res
temp = pd.DataFrame(res)
temp.value_counts()

1000


100%|██████████| 1000/1000 [00:00<00:00, 4490.13it/s]


vader_neu    856
vader_pos    125
vader_neg     19
dtype: int64

In [6]:
new_df['vader_sentiment'] = res
new_df

C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_916\4263305915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['vader_sentiment'] = res


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,vader_sentiment
0,en_0498956,product_en_0828306,reviewer_en_0523960,1,fit well clunki,one star,en,drugstore,vader_pos
1,en_0093747,product_en_0552817,reviewer_en_0994139,1,particular pattern keep get larg hole return f...,one differ print great hole,en,apparel,vader_neu
2,en_0395419,product_en_0647135,reviewer_en_0938679,4,book help learn sexual attitud toward differ r...,book help learn sexual,en,digital_ebook_purchase,vader_neu
3,en_0076703,product_en_0528027,reviewer_en_0988264,2,much smaller anticip take blame assum would no...,disappoint small,en,furniture,vader_neu
4,en_0503361,product_en_0395277,reviewer_en_0258043,2,ring part brome month reveal made cheap materi...,cheap,en,beauty,vader_neu
...,...,...,...,...,...,...,...,...,...
995,en_0348579,product_en_0470247,reviewer_en_0829023,4,nice book want learn journey got gift,worth learn,en,book,vader_pos
996,en_0468187,product_en_0431952,reviewer_en_0025350,5,vibrant fuchsia color make sure order two one ...,love color,en,other,vader_neu
997,en_0065822,product_en_0850981,reviewer_en_0713822,1,order doubl check order like way not call exac...,sent wrong size,en,apparel,vader_neu
998,en_0771285,product_en_0725143,reviewer_en_0038475,1,thing great smaller list say hold much weight ...,not good,en,home,vader_neu


## Roberta

In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

c:\Users\marko.vasilic\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [6]:
def polarity_scores_roberta(text):
    encoded_text = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [10]:
res = []
for index, row in tqdm(new_df.iterrows(), total=len(new_df)):
    temp = polarity_scores_roberta(row['review_body'])
    for key, value in temp.items():
        print(i)
        if value == max(temp['roberta_pos'], temp['roberta_neg'], temp['roberta_neu']):
            res.append(key)
temp = pd.DataFrame(res)
temp.value_counts()

  0%|          | 3/1000 [00:00<01:25, 11.68it/s]

0
0
0
1
1
1
2
2
2
3
3
3


  1%|          | 7/1000 [00:00<01:13, 13.45it/s]

4
4
4
5
5
5
6
6
6
7
7
7


  1%|          | 11/1000 [00:00<01:05, 15.10it/s]

...

98
98
98
99
99
99
100
100
100
101
101
101


 10%|█         | 105/1000 [00:07<01:02, 14.43it/s]

102
102
102
103
103
103
104
104
104
105
105
105


 11%|█         | 106/1000 [00:07<01:02, 14.30it/s]


KeyboardInterrupt: 

In [11]:
new_df['roberta_sentiment'] = res
new_df

C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_916\1791077146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['roberta_sentiment'] = res


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,vader_sentiment,roberta_sentiment
0,en_0498956,product_en_0828306,reviewer_en_0523960,1,fit well clunki,one star,en,drugstore,vader_pos,roberta_neu
1,en_0093747,product_en_0552817,reviewer_en_0994139,1,particular pattern keep get larg hole return f...,one differ print great hole,en,apparel,vader_neu,roberta_neu
2,en_0395419,product_en_0647135,reviewer_en_0938679,4,book help learn sexual attitud toward differ r...,book help learn sexual,en,digital_ebook_purchase,vader_neu,roberta_neu
3,en_0076703,product_en_0528027,reviewer_en_0988264,2,much smaller anticip take blame assum would no...,disappoint small,en,furniture,vader_neu,roberta_neu
4,en_0503361,product_en_0395277,reviewer_en_0258043,2,ring part brome month reveal made cheap materi...,cheap,en,beauty,vader_neu,roberta_neu
...,...,...,...,...,...,...,...,...,...,...
995,en_0348579,product_en_0470247,reviewer_en_0829023,4,nice book want learn journey got gift,worth learn,en,book,vader_pos,roberta_pos
996,en_0468187,product_en_0431952,reviewer_en_0025350,5,vibrant fuchsia color make sure order two one ...,love color,en,other,vader_neu,roberta_neu
997,en_0065822,product_en_0850981,reviewer_en_0713822,1,order doubl check order like way not call exac...,sent wrong size,en,apparel,vader_neu,roberta_neu
998,en_0771285,product_en_0725143,reviewer_en_0038475,1,thing great smaller list say hold much weight ...,not good,en,home,vader_neu,roberta_neu


## Distilbert

In [12]:
from transformers import pipeline

In [13]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

classifier = pipeline("sentiment-analysis", model = model_name)
print(classifier(new_df['review_body'][997])[0]['label'])
print(new_df['review_body'][997])

NEGATIVE
order doubl check order like way not call exact size ship list say actual tripl check made sure order xxl wife l daughter reciev medium larg not order medium extrem leari follow better judgement could not see actual size cart gut feel correct jacket daughter tight hassl return whatev


In [14]:
distilbert_sentiment = []
for index, row in tqdm(new_df.iterrows(), total=len(new_df)):
    res = classifier(row['review_body'])[0]
    if res['score'] > 0.7 and res['label'] == 'POSITIVE':
        distilbert_sentiment.append('distilbert_pos')
    elif res['score'] > 0.7 and res['label'] == 'NEGATIVE':
        distilbert_sentiment.append('distilbert_neg')
    else:
        distilbert_sentiment.append('distilbert_neu')
new_df['distilbert_sentiment'] = distilbert_sentiment

100%|██████████| 1000/1000 [00:19<00:00, 50.82it/s]
C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_916\2621103644.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['distilbert_sentiment'] = distilbert_sentiment


## Calculate score

In [15]:
def check_sentiment(eval, star):
    if (star < 3 and eval != 'neg') or (star > 3 and eval != 'pos') or (star == 3 and eval != 'neu'):
        return False
    return True

In [16]:
vader_score = []
roberta_score = []
distilbert_score = []
for index, row in tqdm(new_df.iterrows(), total=len(new_df)):
    vader_score.append(check_sentiment(row['vader_sentiment'][-3:], row['stars']))
    roberta_score.append(check_sentiment(row['roberta_sentiment'][-3:], row['stars']))
    distilbert_score.append(check_sentiment(row['distilbert_sentiment'][-3:], row['stars']))
new_df['vader_score'] = vader_score
new_df['roberta_score'] = roberta_score
new_df['distilbert_score'] = distilbert_score
print(sum(vader_score)/len(new_df))
print(sum(roberta_score)/len(new_df))
print(sum(distilbert_score)/len(new_df))
new_df

100%|██████████| 1000/1000 [00:00<00:00, 22734.71it/s]

0.274
0.481
0.57



C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_916\3076350570.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['vader_score'] = vader_score
C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_916\3076350570.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['roberta_score'] = roberta_score
C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_916\3076350570.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,vader_sentiment,roberta_sentiment,distilbert_sentiment,vader_score,roberta_score,distilbert_score
0,en_0498956,product_en_0828306,reviewer_en_0523960,1,fit well clunki,one star,en,drugstore,vader_pos,roberta_neu,distilbert_pos,False,False,False
1,en_0093747,product_en_0552817,reviewer_en_0994139,1,particular pattern keep get larg hole return f...,one differ print great hole,en,apparel,vader_neu,roberta_neu,distilbert_neg,False,False,True
2,en_0395419,product_en_0647135,reviewer_en_0938679,4,book help learn sexual attitud toward differ r...,book help learn sexual,en,digital_ebook_purchase,vader_neu,roberta_neu,distilbert_neg,False,False,False
3,en_0076703,product_en_0528027,reviewer_en_0988264,2,much smaller anticip take blame assum would no...,disappoint small,en,furniture,vader_neu,roberta_neu,distilbert_neg,False,False,True
4,en_0503361,product_en_0395277,reviewer_en_0258043,2,ring part brome month reveal made cheap materi...,cheap,en,beauty,vader_neu,roberta_neu,distilbert_neg,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,en_0348579,product_en_0470247,reviewer_en_0829023,4,nice book want learn journey got gift,worth learn,en,book,vader_pos,roberta_pos,distilbert_pos,True,True,True
996,en_0468187,product_en_0431952,reviewer_en_0025350,5,vibrant fuchsia color make sure order two one ...,love color,en,other,vader_neu,roberta_neu,distilbert_pos,False,False,True
997,en_0065822,product_en_0850981,reviewer_en_0713822,1,order doubl check order like way not call exac...,sent wrong size,en,apparel,vader_neu,roberta_neu,distilbert_neg,False,False,True
998,en_0771285,product_en_0725143,reviewer_en_0038475,1,thing great smaller list say hold much weight ...,not good,en,home,vader_neu,roberta_neu,distilbert_neg,False,False,True


## Vocabulary sentiment analysis

In [17]:
org_df = pd.read_csv("..\data\smart_store_reviews.csv")
org_df = org_df.sample(frac=1).reset_index(drop=True)
new_org_df = org_df.iloc[:1000, :]
pos_words = pd.read_table('..\data\positive-words.txt', delimiter=" ")
neg_words = pd.read_table('..\data\\negative-words.txt', delimiter=" ")
pos_words_set = set(pos_words['Words'])
neg_words_set = set(neg_words['Words'])

In [18]:
def add_eval(pos_counter, neg_counter):
    if pos_counter > neg_counter:
        return 'vocabulary_pos'
    elif pos_counter == neg_counter:
        return 'vocabulary_neu'
    else:
        return 'vocabulary_neg'

In [19]:
voc_eval = []
for index, row in tqdm(new_org_df.iterrows(), total=len(new_org_df)):
    splits = row['review_body'].split()
    pos_counter = 0
    neg_counter = 0
    for s in splits:
        if (s in pos_words_set) == True:
            pos_counter += 1
        elif (s in neg_words_set) == True:
            neg_counter += 1
    voc_eval.append(add_eval(pos_counter, neg_counter))
new_org_df['voc_eval'] = voc_eval
new_org_df
        

100%|██████████| 1000/1000 [00:00<00:00, 24999.28it/s]
C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_916\940737511.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_org_df['voc_eval'] = voc_eval


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,voc_eval
0,en_0384645,product_en_0637944,reviewer_en_0397189,4,"These socks have good compression, but I'm not...",does what they are designed for,en,apparel,vocabulary_neg
1,en_0989120,product_en_0894805,reviewer_en_0936807,1,This product said it was delivered on Nov. 9 t...,Not Received,en,jewelry,vocabulary_neu
2,en_0474571,product_en_0512133,reviewer_en_0005993,3,I wouldn't buy this again. The quality if fair...,Meh not worth the money or time to assemble,en,home,vocabulary_neu
3,en_0375355,product_en_0099049,reviewer_en_0558683,4,Our Mastiff loves it and puts it on easily.,Doggie shirt,en,pet_products,vocabulary_pos
4,en_0200169,product_en_0298055,reviewer_en_0009363,2,"My 2.5 year old toddler is a bit small, so the...",Run big,en,apparel,vocabulary_neu
...,...,...,...,...,...,...,...,...,...
995,en_0208902,product_en_0466926,reviewer_en_0224417,2,my dog don't like it,Two Stars,en,pet_products,vocabulary_pos
996,en_0477397,product_en_0164907,reviewer_en_0790188,3,I picked this up specifically for the usb-c co...,usb-c cord doesn't work,en,home,vocabulary_neu
997,en_0120877,product_en_0327473,reviewer_en_0948340,1,Garbage. This is very poorly made that doesn’t...,Very low quality product even for the low price.,en,home_improvement,vocabulary_neg
998,en_0542707,product_en_0651972,reviewer_en_0456560,1,The bag with the hoes was wrinkled and not sea...,Received a used item...yuk,en,drugstore,vocabulary_neg


In [20]:
voc_score = []
for index, row in tqdm(new_org_df.iterrows(), total=len(new_org_df)):
    voc_score.append(check_sentiment(row['voc_eval'][-3:], row['stars']))
new_org_df['voc_score'] = voc_score
sum(voc_score)/len(new_org_df)

100%|██████████| 1000/1000 [00:00<00:00, 31221.79it/s]
C:\Users\marko.vasilic\AppData\Local\Temp\ipykernel_916\2385608508.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_org_df['voc_score'] = voc_score


0.46